In [2]:
import pandas as pd
import numpy as np
import regex as re

import openai
import pickle
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [8]:
df = pd.read_csv("./data/amazon.csv")
df.head(3)

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...


## Data Preprocessing 

In [32]:
df = df[["product_name", "category", "about_product", "review_title"]]
df.sample(300)
df.head(1)

,product_name,category,about_product,review_title
0,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers and Accessories Accessories and Peri...,High Compatibility Compatible With iPhone 12 1...,SatisfiedCharging is really fastValue for mone...


In [33]:
def text_cleanup(text):
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\&", " and ", text)
    text = re.sub(r"\|", " ", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[^\w\d\s]", "", text)
    return text.strip()

In [34]:
# List of columns to clean
columns_to_clean = ["product_name", "category", "about_product", "review_title"]

# Apply text_cleanup function to each specified column
df[columns_to_clean] = df[columns_to_clean].map(text_cleanup)

In [35]:
for _, row in df.head(3).iterrows():
    print("Product name:", row["product_name"], "\n")
    print("Category:", row["category"], "\n")
    print("Product Description:", row["about_product"], "\n")
    print("Review Title:", row["review_title"], "\n")
    print('----------')

Product name: Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13 1211 X 8 7 6 5 iPad Air Pro Mini 3 FT Pack of 1 Grey 

Category: Computers and Accessories Accessories and Peripherals Cables and Accessories Cables USBCables 

Product Description: High Compatibility Compatible With iPhone 12 11 XXsMaxXr iPhone 88 PlusiPhone 77 PlusiPhone 6s6s PlusiPhone 66 PlusiPhone 55s5cseiPad ProiPad Air 12iPad mini 123iPod nano7iPod touch and more apple devices Fast Charge and Data Sync It can charge and sync simultaneously at a rapid speed Compatible with any charging adaptor multiport charging station or power bank Durability Durable nylon braided design with premium aluminum housing and toughened nylon fiber wound tightly around the cord lending it superior durability and adding a bit to its flexibility High Security Level It is designed to fully protect your device from damaging excessive currentCopper core thickMultilayer shielding Antiinterference 